В этом блокноте я предпринял попытку улучшить общую модель исправления ошибок: научиться исправлять ошибки на "не/ни". Хотел попробовать и опечатки, но не успел. 

Как это было сделано:

* Чтобы не возиться с машинным обучением, скачал готовые словные N-граммы из НКРЯ, и построил из них языковую модель - да-да, старую добрую "статистическую" языковую модель со сглаживанием. Нечто подобное я пробовал ещё два года назад (https://habr.com/ru/post/346578/), и оно зашло норм даже после обучения на неуместном корпусе. А НКРЯ, кажется, достаточно литературен, чтобы быть похожим на выборку из этой конкретной задачи. 
* Языковой моделью я скорил гипотезы, генерируемые вручную: "не" и "ни" замеяются друг на друга, отклеиваются от следующего слова или приклеиваются к нему. Попробовал сделать ещё генерилку всех редактур отдельного слова, соответствующих изменению одного символа, но не успел засабмитить. 

In [0]:
!wget  https://github.com/datasouls/nlp-hack-2019/raw/master/data/grammar_train.csv

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_orig = pd.read_csv('grammar_train.csv')

In [0]:
pd.options.display.max_colwidth = 300
train_orig

,id,correct_sentence,sentence_with_a_mistake,mistake_type
0,0,И сиял оскал его белее того снега!,И сиял оскал его белее того снега!,нет ошибки
1,1,"Я заметил, что наш кулич был гораздо белее того, какой был у соседей.","Я заметил, что наш кулич был гораздо белее того, какой был у соседей.",нет ошибки
2,2,Но сегодня ей предстояло везти сына на тренировку по футболу.,Но сегодня ей предстояло везти сына на тренировку по футболу.,нет ошибки
3,3,"Если делать и то и другое, то всё получится плохо.","Если делать и то и другое, то всё получиться плохо.",тся-ться
4,4,Все удалится без пересборки индекса.,Все удалиться без пересборки индекса.,тся-ться
5,5,Белки должны увеличиться почти в 2 раза.,Белки должны увеличится почти в 2 раза.,тся-ться
6,6,"Очевидно, им нравится подвергать себя опасности.","Очевидно, им нравиться подвергать себя опасности.",тся-ться
7,7,"Впрочем, чтобы разобраться, на каком слоге стоит ударение в каждом случае, приходится тщательно вслушиваться.","Впрочем, чтобы разобраться, на каком слоге стоит ударение в каждом случае, приходиться тщательно вслушиваться.",тся-ться
8,8,К нему надо как к вазе относиться.,К нему надо как к вазе относится.,тся-ться
9,9,"Да не надо тебе краситься, вот лишь бы что-нибудь сделать с собой.","Да не надо тебе красится, вот лишь бы что-нибудь сделать с собой.",тся-ться


http://www.ruscorpora.ru/new/corpora-freq.html

In [0]:
!wget http://www.ruscorpora.ru/new/ngrams/1grams-3.zip
!wget http://www.ruscorpora.ru/new/ngrams/2grams-3.zip
!wget http://www.ruscorpora.ru/new/ngrams/3grams-3.zip
!wget http://www.ruscorpora.ru/new/ngrams/4grams-2.zip
!wget http://www.ruscorpora.ru/new/ngrams/5grams-2.zip

In [0]:
!unzip 1grams-3.zip
!unzip 2grams-3.zip
!unzip 3grams-3.zip
!unzip 4grams-2.zip
!unzip 5grams-2.zip

Archive:  1grams-3.zip
  inflating: 1grams-3.txt            
Archive:  2grams-3.zip
  inflating: 2grams-3.txt            
Archive:  3grams-3.zip
  inflating: 3grams-3.txt            
Archive:  4grams-2.zip
  inflating: 4grams-2.txt            
Archive:  5grams-2.zip
  inflating: 5grams-2.txt            


In [0]:
!ls

1grams-3.txt  2grams-3.zip  3grams-3.zip  5grams-2.txt	     sample_data
1grams-3.zip  2HDwUBO	    4grams-2.txt  5grams-2.zip	     tsya_example.csv
2grams-3.txt  3grams-3.txt  4grams-2.zip  grammar_train.csv


In [0]:
p = '.'
import os

In [0]:
os.listdir(p)
grams_lines = {}
for fn in os.listdir(p):
    if not fn.endswith('txt'):
        continue
    print(fn)
    f0 = int(fn[0])
    with open(os.path.join(p, fn), encoding='utf-8') as f:
        grams_lines[f0] = [l.split('\t') for l in f.readlines()]

5grams-2.txt
3grams-3.txt
1grams-3.txt
4grams-2.txt
2grams-3.txt


In [0]:
from collections import Counter

In [0]:
from tqdm.auto import tqdm, trange

In [0]:
def get_ngram(l):
    l = [w.strip() for w in l[1:]]
    return '_'.join([w for w in l if w])

In [0]:
del grams_lines

Создаём словарики, мапящие n-грамму в её частоту. 

In [0]:
grams_freq = {
    n: {
        get_ngram(l): int(l[0])
        for l in d
    }
    for n, d in grams_lines.items()
}

In [0]:
grams_freq[0] = {'': sum(grams_freq[1].values())}

In [0]:
from copy import deepcopy

In [0]:
np.mean([n >= 10 for n in grams_freq[3].values()])

0.15801356341444028

Чтобы получившаяся моделька была полегче, в итоге я решил обойтись 1-3 граммами, и взял только те, которые встречались в НКРЯ хотя бы 10 раз. Понятно, что это просаживает качество, но важнее было уложиться в лимиты по времени и памяти. 

In [0]:
gf3 = {n: {k: v for k, v in grams_freq[n].items() if v >=10 } for n in range(4)}

In [0]:
import pickle
with open('gf3.pkl', 'wb') as f:
    pickle.dump(gf3, f)

In [0]:
toks = 'Дело тут ни в их нежелании работать или лени'.lower().split()
print(toks)
toks2 = toks[:]
toks2[2] = 'не'
print(toks2)

['дело', 'тут', 'ни', 'в', 'их', 'нежелании', 'работать', 'или', 'лени']
['дело', 'тут', 'не', 'в', 'их', 'нежелании', 'работать', 'или', 'лени']


In [0]:
from copy import copy

Эта функция создаёт гипотезы, как можно изменить "не" и "ни" в предложении. 

In [0]:
def make_hypotheses(tokens, result=None, max_changes=2):
    if max_changes <= 0:
        return
    tokens = tuple(tokens)
    if result is None:
        result = {tuple(tokens)}
    elif tokens in result:
        return
    result.add(tokens)
    for i, token in enumerate(tokens):
        if token == 'не' or token == 'ни':
            make_hypotheses(tuple(tokens[:i] + ('ни',) + tokens[(i+1):]), result=result, max_changes=max_changes-1)
            make_hypotheses(tuple(tokens[:i] + ('не',) + tokens[(i+1):]), result=result, max_changes=max_changes-1)
            if i + 1 < len(tokens) and len(tokens[i+1]) >= 2:  # exclude 'нив'
                make_hypotheses(tuple(tokens[:i] + ('ни' + tokens[i+1],) + tokens[(i+2):]), result=result, max_changes=max_changes-1)
                make_hypotheses(tuple(tokens[:i] + ('не' + tokens[i+1],) + tokens[(i+2):]), result=result, max_changes=max_changes-1)
        elif token.startswith('не') or token.startswith('ни'):
             make_hypotheses(tuple(tokens[:i] + ('не', token[2:]) + tokens[(i+1):]), result=result, max_changes=max_changes-1)
             make_hypotheses(tuple(tokens[:i] + ('ни', token[2:]) + tokens[(i+1):]), result=result, max_changes=max_changes-1)
    return result

len(make_hypotheses(tuple(toks)))

14

In [0]:
gf3.keys()

dict_keys([0, 1, 2, 3])

А эта функция высчитывает вероятность каждого предложения по n-грамной языковой модели. 

Я смешиваю n c 1 до N, накапливая числители и знаменатели для разных n с соответствующими коэффициентами. Коэффициенты точно стоило потюнить на каком-нибудь датасете, но я как-то не догадался, и ограничился интуицией и ручным подкручиванием. 

In [0]:
no_count = 1e-10
no_count2 = 1.0

order_weights = [None, 0.001, 0.01, 100, 1000, 10000]


def get_log_proba(toks, order=3, prnt=False, grams=gf3):
    log_proba = 1.0
    ps = []
    for i, word in enumerate(toks):
        num = 1e-20
        den = 1e-10
        for n in range(1, order+1):
            if i+1 >= n:
                num += order_weights[n] * grams[n].get('_'.join(toks[(i-n+1):(i+1)]), no_count)
                den += order_weights[n] * grams[n-1].get('_'.join(toks[(i-n+1):(i)]), no_count2)    
        p = num / den
        if prnt:
            print(num, den, '_'.join(toks[(i-1+1):(i+1)]), p)
        ps.append(np.log(p)) 
        log_proba += np.log(p)
    if prnt:
        print(ps)
    return log_proba

# print(get_log_proba(toks, prnt=True))
print(get_log_proba('людей ни что неучит'.split(), prnt=True))

100.77 190827.1740000001 людей 0.0005280694457069304
286.224 191834.8740000001 ни 0.001492033195173886
2066.03800001 199983.1140000001 что 0.0103310622516359
1.000110000001e-08 230068.95400000009 неучит 4.347001116895588e-14
[-7.546282756946698, -6.507615528672005, -4.572600169420477, -30.766705092975215]
-48.3932035480144


In [0]:
!pip install textdistance

In [0]:
import textdistance

In [0]:
textdistance.levenshtein('не в', 'нив')

2

In [0]:
neni = train_orig[train_orig.mistake_type.str.contains('не.ни')]
neni


,id,correct_sentence,sentence_with_a_mistake,mistake_type
30,30,Дело тут не в их нежелании работать или лени.,Дело тут ни в их нежелании работать или лени.,не/ни-1
31,31,"Прошло много лет, но он нисколько не изменился.","Прошло много лет, но он несколько не изменился.",не/ни-2
32,32,"Никто не может противостоять тому, что прописано судьбой, дорогая.","Не кто не может противостоять тому, что прописано судьбой, дорогая.",не/ни-3
33,33,Людей ничто не учит!,Людей ни что не учит!,"не,ни-4"
34,34,Не давайте никому морочить вам голову,Не давайте ни кому морочить вам голову,"не,ни-4"


In [382]:
this = train_orig[train_orig.mistake_type.str.contains('то')]
this

,id,correct_sentence,sentence_with_a_mistake,mistake_type
35,35,Нельзя надевать одно и то же для повседневной работы и для важной встречи.,Нельзя надевать одно и тоже для повседневной работы и для важной встречи.,"тоже, чтоб"
36,36,В одно и то же озеро можно заходить и выходить хоть тысячу раз.,В одно и тоже озеро можно заходить и выходить хоть тысячу раз.,"тоже, чтоб"
38,38,"С остальными, наверное, как-то так же.","С остальными, наверное, как то так же.","то, таки"


In [0]:
not_neni = train_orig[~train_orig.mistake_type.str.contains('не.ни')]

In [0]:
from nltk import wordpunct_tokenize

Одного скора языковой модели недостаточно: надо проверять, насколько результирующее предложение далеко от оригинального, т.к. вообще-то грамматически ошибки - явление очень редкое, и обычно не случается много их. Я подобрал отдельный коэффициент, штрафующий за каждый балл расстояния Левенштейна между старым и новым текстом. 

In [0]:
def rank_hypotheses(toks, hypotheses, grams, leven_penalty=0.2, order=5):
    scores = []
    for hypo in hypotheses:
        scores.append([' '.join(hypo), get_log_proba(hypo, order=order, grams=grams), len(hypo)])

    d = pd.DataFrame(scores)
    d.columns = ['text', 'lm', 'n']
    d['leven'] = d.text.apply(lambda x: textdistance.levenshtein(x, ' '.join(toks)))
    d['penalty'] = np.log(leven_penalty) * d.leven
    d['score'] = d.lm + d.penalty
    d.sort_values('score', ascending=False, inplace=True)
    return d



In [373]:
toks0 = wordpunct_tokenize(neni.iloc[4].sentence_with_a_mistake)
toks = [t.lower() for t in toks0 if t.isalpha()]
print(toks)

hypos = make_hypotheses(tuple(toks), max_changes=3)
d = rank_hypotheses(toks, hypos, gf3, leven_penalty=0.15, order=3)

print(d.iloc[0].score - d.iloc[1].score)
d

['не', 'давайте', 'ни', 'кому', 'морочить', 'вам', 'голову']
3.543903475834476


,text,lm,n,leven,penalty,score
10,не давайте никому морочить вам голову,-52.511615,6,1,-1.89712,-54.408735
4,не давайте некому морочить вам голову,-54.158399,6,2,-3.79424,-57.952639
3,не давайте не кому морочить вам голову,-56.555397,7,1,-1.89712,-58.452517
7,не давайте ни кому морочить вам голову,-58.788911,7,0,-0.00000,-58.788911
8,ни давайте никому морочить вам голову,-55.058083,6,2,-3.79424,-58.852322
11,ни давайте некому морочить вам голову,-56.704866,6,3,-5.69136,-62.396226
5,ни давайте не кому морочить вам голову,-59.101864,7,2,-3.79424,-62.896104
6,ни давайте ни кому морочить вам голову,-61.335378,7,1,-1.89712,-63.232498
9,недавайте не кому морочить вам голову,-83.863280,6,2,-3.79424,-87.657520
1,недавайте ни кому морочить вам голову,-86.130126,6,1,-1.89712,-88.027246


In [0]:
pd.options.display.max_colwidth = 300

Похожий механизм попробовали сделать для ошибок типа "тоже", "также", "чтобы", но он с ходу не очень хорошо полетел. 

In [0]:
def iter_hypotheses_what(tokens):
    pos = 0
    while pos < len(tokens):
        token = tokens[pos]
        if token.lower() in {'тоже', 'также'}:
            yield (tokens[:pos]
                   + [token[:-2]] + ['же']
                   + tokens[pos + 1:])
            pos += 1
        elif token.lower() == 'чтоб':
            yield (tokens[:pos]
                   + [token[:-1]] + ['б']
                   + tokens[pos + 1:])
            pos += 1
        elif token.lower() == 'чтобы':
            yield (tokens[:pos]
                   + [token[:-2]] + ['бы']
                   + tokens[pos + 1:])
            pos += 1
        elif pos + 1 < len(tokens) and token.lower() in {'то', 'так'} and tokens[pos + 1] == 'же':
            yield (tokens[:pos]
                   + [token + 'же']
                   + tokens[pos + 2:])
            pos += 2
        elif pos + 1 < len(tokens) and token.lower() == 'что' and tokens[pos + 1] in {'б', 'бы'}:
            yield (tokens[:pos]
                   + [token + tokens[pos + 1]]
                   + tokens[pos + 2:])
            pos += 2
        else:
            pos += 1

def make_hypotheses_what(tokens):
    hypos = [tuple(tokens)]
    for hypo in iter_hypotheses_what(tokens):
        hypo = tuple(hypo)
        if hypo not in hypos:
            hypos.append(hypo)
    return hypos

Тут я валидировал, что на ошибках типа "не ни" моя модель возвращает правильный ответ, а на других ошибках и правильных предложениях  - не меняет ничего. 

In [411]:
for i, row in tqdm(this.iterrows()):
    sent = row.sentence_with_a_mistake
    sentc = row.correct_sentence
    toks0 = wordpunct_tokenize(sent)
    toks = [t.lower() for t in toks0 if t.isalpha()]
    hypos = make_hypotheses(tuple(toks), max_changes=3)
    #hypos = make_hypotheses_what(toks)
    print(len(hypos))
    # 0.15 is too strict
    d = rank_hypotheses(toks, hypos, grams=gf3, leven_penalty=0.16, order=3)
    if not d.iloc[0].leven == 0:
        pass
        #print(d.head(3))
    new_text = d.text.iloc[0]
    #print(toks0)
    #print(new_text.split())
    result = denormalize(sent, toks0, new_text.split())
    #assert result == sent
    print(result == sentc)
    #print(result)

2
True
2
False
2
False


Очень важная задача: восстановить оригинальное предложение! Я применял языковую модель к токенизированному предложению, с потерей капитализации и знаков препинания, но вернуть-то надо оригинальное предложение, только с исправленными ошибками. Поэтому я написал функцию, просматривающую старые и новые токены, выстраивающую соответствие между ними, и воссоздающую новое предложение с учётом специфики старых токенов и исправлений, появившихся в новых. 

Потом уже товарищи мне подсказали, что вместо стрёмного токенизатора из nltk можно использовать razdel, но времени менять токенизатор и прочее уже особо не было. 

In [364]:
def denormalize(orig, toks0, toks):
    result = ' '
    i0 = 0
    i1 = 0
    while i0 < len(toks0):
        #print('"{}  {} {}" "{}" '.format(result, i0, i1, toks0[i0]))
        prev_is_punct = result[-1] in '.?-:!'
        if i0 < len(toks0) and i1 < len(toks) and toks0[i0].lower() == toks[i1].lower():
            if not prev_is_punct:
                result = result + ' ' 
            result = result + toks0[i0]
            i0 += 1
            i1 += 1
        elif not toks0[i0].isalpha():
            if toks0[i0].isnumeric():
                if not prev_is_punct:
                    result = result + ' ' 
            result = result + toks0[i0]
            i0 += 1
        elif i0 < len(toks0) and i1 < len(toks):
            #print('change "{}" -> "{}"'.format(toks0[i0], toks[i1]))
            if abs(len(toks0[i0]) - len(toks[i1])) <= 1: # probably, the same token, but with correction
                tok = toks[i1]
                if toks0[i0][0].isupper():
                    tok = tok.capitalize()
                if not prev_is_punct:
                    result = result + ' ' 
                    result = result + tok
                i0 += 1
                i1 += 1
            elif len(toks0[i0]) > len(toks[i1]):
                tok = toks[i1] + ' ' + toks[i1+1]
                if toks0[i0][0].isupper():
                    tok = tok.capitalize()
                if not prev_is_punct:
                    result = result + ' ' 
                    result = result + tok
                i0 += 1
                i1 += 2
            else:
                tok = toks[i1]
                if toks0[i0][0].isupper():
                    tok = tok.capitalize()
                if not prev_is_punct:
                    result = result + ' ' 
                    result = result + tok
                i0 += 2
                i1 += 1
        else:
            raise ValueError('{} is wrong with i0={}, i1={}'.format(result, i0, i1))
    result = result.strip()
    while orig.endswith(' '):
        orig = orig[:(-1)]
        result = result + ' '
    return result

result = denormalize(sent, toks0, toks)
print(sent)
print(result)
assert result == sent

Не давайте ни кому морочить вам голову
Не давайте ни кому морочить вам голову


Теперь можно написать функцию, которая принимает исходный текст, генераторы гипотез и языковую модель - и возвращает исправленные предложения!

In [0]:
def make_correction(text, hypo_makers, grams):
    toks0 = wordpunct_tokenize(text)
    toks = [t.lower() for t in toks0 if t.isalpha()]
    hypos = [toks]
    for maker in hypo_makers:
        hypos.extend(maker(toks))
    d = rank_hypotheses(toks, hypos, grams=grams, leven_penalty=0.15, order=3, )
    new_text = d.text.iloc[0]
    result = denormalize(text, toks0, new_text.split())
    return result

In [381]:
print(make_correction('Меня ни кто нелюбит, Антон!!', [make_hypotheses], gf3))

Меня ни кто не любит, Антон!!


## Опечатки

А тут я было начал адаптировать код Норвига (norvig.com/spell-correct.html) для работы с опечатками, но уже не успел до конца соревнования - начались проблемы с заливкой моего решения для не/ни, и в итоге оно так и не успело докатиться до лидерборда. Впрочем, это не помешало нам победить. 

In [0]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [0]:

unigramset = set(gf3[1].keys())

In [0]:
def make_hypotheses_list(tokens, unigramset):
    hypos = [tuple(tokens)]
    for i, tok in enumerate(tokens):
        for edit in edits1(tok):
            if edit in unigramset:
                hypo = copy(tokens)
                hypo[i] = edit
                if hypo not in hypos:
                    hypos.append(hypo)
    return hypos

In [432]:
toks = 'белее того это не принято в цивилизованном обществе'.split()
hypos = make_hypotheses_list(toks, unigramset)
print(len(hypos))

202


Из коробки эта корректировалка довольно прилично работает с произвольно выбранной опечаткой - 

In [433]:
d = rank_hypotheses(toks, hypos, gf3, leven_penalty=0.15, order=3)
d.head(10)

,text,lm,n,leven,penalty,score
5,более того это не принято в цивилизованном обществе,-50.198944,8,1,-1.89712,-52.096064
6,белые того это не принято в цивилизованном обществе,-52.423425,8,1,-1.89712,-54.320545
3,белье того это не принято в цивилизованном обществе,-53.225041,8,1,-1.89712,-55.122161
10,белое того это не принято в цивилизованном обществе,-53.594059,8,1,-1.89712,-55.491179
0,белее того это не принято в цивилизованном обществе,-56.061224,8,0,-0.00000,-56.061224
8,белее того это не принято в цивилизованном обществе,-56.061224,8,0,-0.00000,-56.061224
9,белеет того это не принято в цивилизованном обществе,-56.012065,8,1,-1.89712,-57.909185
1,белел того это не принято в цивилизованном обществе,-56.076849,8,1,-1.89712,-57.973969
23,белее этого это не принято в цивилизованном обществе,-56.087122,8,1,-1.89712,-57.984242
2,белке того это не принято в цивилизованном обществе,-56.686924,8,1,-1.89712,-58.584044


In [434]:
toks = 'он считается без везти пропавшим'.split()
hypos = make_hypotheses_list(toks, unigramset)
print(len(hypos))
d = rank_hypotheses(toks, hypos, gf3, leven_penalty=0.15, order=3)
d.head(10)

109


,text,lm,n,leven,penalty,score
94,он считается без вести пропавшим,-35.880480,5,1,-1.89712,-37.777600
0,он считается без везти пропавшим,-48.180456,5,0,-0.00000,-48.180456
28,он считается без везти пропавшим,-48.180456,5,0,-0.00000,-48.180456
102,он считается без везти пропавших,-46.748779,5,1,-1.89712,-48.645899
106,он считается без везти попавшим,-47.353778,5,1,-1.89712,-49.250898
104,он считается без везти пропавшие,-47.907163,5,1,-1.89712,-49.804283
98,он считается без везли пропавшим,-48.258020,5,1,-1.89712,-50.155140
103,он считается без везти пропавшем,-48.301817,5,1,-1.89712,-50.198937
107,он считается без везти пропавшими,-48.600310,5,1,-1.89712,-50.497430
19,от считается без везти пропавшим,-48.641447,5,1,-1.89712,-50.538567


In [435]:
toks = 'вероятно оба вши предположения не лишены основания'.split()
hypos = make_hypotheses_list(toks, unigramset)
print(len(hypos))
d = rank_hypotheses(toks, hypos, gf3, leven_penalty=0.15, order=3)
d.head(10)

137


,text,lm,n,leven,penalty,score
42,вероятно оба ваши предположения не лишены основания,-63.916029,7,1,-1.89712,-65.813149
44,вероятно оба уши предположения не лишены основания,-64.428327,7,1,-1.89712,-66.325447
8,вероятно она вши предположения не лишены основания,-64.655173,7,1,-1.89712,-66.552293
0,вероятно оба вши предположения не лишены основания,-67.719026,7,0,-0.00000,-67.719026
1,вероятно оба вши предположения не лишены основания,-67.719026,7,0,-0.00000,-67.719026
13,вероятно об вши предположения не лишены основания,-65.883640,7,1,-1.89712,-67.780760
39,вероятно оба вси предположения не лишены основания,-67.140787,7,1,-1.89712,-69.037907
50,вероятно оба вши предположение не лишены основания,-67.402523,7,1,-1.89712,-69.299643
136,вероятно оба вши предположения не лишены основании,-67.497549,7,1,-1.89712,-69.394669
126,вероятно оба вши предположения не лишен основания,-67.560559,7,1,-1.89712,-69.457678


А вот на этом примере моя моделька, увы, предсказала не то, что нужно: правильная гипотеза оказалась только третьей по счёту. 

In [441]:
toks = 'в результате получатся удлиненные трубочки из которых уже можно плести бес проблем'.split()
hypos = make_hypotheses_list(toks, unigramset)
print(len(hypos))
d = rank_hypotheses(toks, hypos, gf3, leven_penalty=0.15, order=3)
d.head(10)

210


,text,lm,n,leven,penalty,score
70,в результате получается удлиненные трубочки из которых уже можно плести бес проблем,-103.298879,12,1,-1.89712,-105.195999
68,в результате получится удлиненные трубочки из которых уже можно плести бес проблем,-104.327811,12,1,-1.89712,-106.224931
173,в результате получатся удлиненные трубочки из которых уже можно плести без проблем,-108.178793,12,1,-1.89712,-110.075913
185,в результате получатся удлиненные трубочки из которых уже можно плести лес проблем,-111.191043,12,1,-1.89712,-113.088163
69,в результате получаются удлиненные трубочки из которых уже можно плести бес проблем,-111.370821,12,1,-1.89712,-113.267941
50,в результате получатся удлиненные трубочки из которых уже можно плести бес проблем,-113.589375,12,0,-0.00000,-113.589375
0,в результате получатся удлиненные трубочки из которых уже можно плести бес проблем,-113.589375,12,0,-0.00000,-113.589375
187,в результате получатся удлиненные трубочки из которых уже можно плести вес проблем,-112.230123,12,1,-1.89712,-114.127243
67,в результате получался удлиненные трубочки из которых уже можно плести бес проблем,-112.516097,12,1,-1.89712,-114.413217
75,в результате получатся удлиненные трубочку из которых уже можно плести бес проблем,-112.744710,12,1,-1.89712,-114.641830
